# Analyzing Learning-Based Networked Systems with Formal Verification

## Experiments and examples

This notebook contains the code used to collect the results shown in section 6 of our paper. To run this notebook, make sure to have installed the requirements as indicated in the README for the selected kernel.

In [ ]:
import os
import sys

import time
import numpy as np

from collections import OrderedDict, defaultdict
from matplotlib import pyplot as plt
from pprint import pprint


import pensieve

## Experiment 1: Resilience to adversarial perturbations

This experiment aims to test how resilient Pensieve is to changes in the inputs. The returned value is $\epsilon$, the attacker power (proportional to the full range of attacked inputs). Two cases are tested: one with constant throughput, and one with an actual trace taken from the Pensieve dataset.

### Implementation

This experiment is implemented by searching for bounds on $\epsilon$ such that the attacker cannot change the output with the lower bound, but can change it with the higher bound.

The result of the experiments is written to the file `test_resilience_pensieve_caseX`.

In [ ]:
def map_to_closest(value, options):
    dist = float('inf')
    selected = None
    for option in options:
        if abs(value - option) < dist:
            selected = option
            dist = abs(option - value)
    return selected


def test_perturbation(default_values, expected_br, delta, epsilon, discrete_inputs):
    # returns true if the perturbation does not cause a violation
    bounds = dict()
    
    for var in default_values:
        lb = default_values[var] - delta[var]*epsilon
        ub = default_values[var] + delta[var]*epsilon
        
        if var in discrete_inputs:
            lb = map_to_closest(lb, discrete_inputs[var])
            ub = map_to_closest(ub, discrete_inputs[var])
            
        bounds[var] = (lb, ub)
    
    # Encoding of the network
    exp = pensieve.PropertyExplainer()
    exp.encode_input_layers()
    exp.encode_hidden_layers()
    
    for var in bounds:
        lb, ub = bounds[var]
        exp.encode_tighter_bounds(var, lb, ub)
        
    exp.encode_expected_output(expected_br, inverted=True)
    
    exp.find_solutions(stop_after_one=True)
    
    return len(exp.solutions) == 0


def search_perturbation_resilience(initial_values,
                                   expected_output,
                                   scaled_deltas,
                                   discrete_inputs,
                                   precision=0.001):
    epsilon = 0
    # check that we actually match expected output with epsilon == 0
    if not test_perturbation(initial_values, expected_output, scaled_deltas, 0, discrete_inputs):
        print('Error: initial values do not match expected output')
        return 0
    
    epsilon = 0.01
    low_bound = 0
    high_bound = None
    start_time = time.time()
    # multiplicative increase search for bounds
    while test_perturbation(initial_values, expected_output, scaled_deltas, epsilon, discrete_inputs):
        low_bound = epsilon
        epsilon *= 1.3
        print(f'searching bounds (epsilon = {epsilon})')
    high_bound = epsilon
    print(f'found bounds: ({low_bound}, {high_bound})')
    
    # binary search for exact value in (low_bound, high_bound)
    while high_bound - low_bound > precision:
        epsilon = (high_bound + low_bound) / 2
        if test_perturbation(initial_values, expected_output, scaled_deltas, epsilon, discrete_inputs):
            low_bound = epsilon
        else:
            high_bound = epsilon 
        print(f'refined bounds: ({low_bound}, {high_bound})')
        
    end_time = time.time()
    print('Run time:', round(end_time - start_time, 3), 'seconds')
            
    return epsilon

## Case 1: Constant throughput of 1850Kbps

The values below represent the default input being tested in case 1.

In [ ]:
default_values = {
    'x_prev_bitrate_0$': 1850/4300, # expect 1850 Kbps
    'x_buffer_0$': 1.573854,
    'x_throughput_0$': 0.23125, # fixed 1850 Kbps goodput
    'x_throughput_1$': 0.23125,
    'x_throughput_2$': 0.23125,
    'x_throughput_3$': 0.23125,
    'x_throughput_4$': 0.23125,
    'x_throughput_5$': 0.23125,
    'x_throughput_6$': 0.23125,
    'x_throughput_7$': 0.23125,
    'x_latency_0$': 0.504, # next_size_3 / throughput / 8
    'x_latency_1$': 0.504,
    'x_latency_2$': 0.504,
    'x_latency_3$': 0.504,
    'x_latency_4$': 0.504,
    'x_latency_5$': 0.504,
    'x_latency_6$': 0.504,
    'x_latency_7$': 0.504,
    'x_next_size_0$': 0.153233,
    'x_next_size_1$': 0.380457,
    'x_next_size_2$': 0.605353,
    'x_next_size_3$': 0.931950,
    'x_next_size_4$': 1.437556,
    'x_next_size_5$': 2.158495,
    'x_remaining_chunks_0$': 24/48,
}

bounds = {
    'x_prev_bitrate_0$': (300/4300, 4300/4300),
    'x_buffer_0$': (0.4, 3.267816),
    'x_throughput_0$': (0, 0.541045),
    'x_throughput_1$': (0, 0.541045),
    'x_throughput_2$': (0, 0.541045),
    'x_throughput_3$': (0, 0.541045),
    'x_throughput_4$': (0, 0.541045),
    'x_throughput_5$': (0, 0.542024),
    'x_throughput_6$': (0, 0.542086),
    'x_throughput_7$': (0.069767, 0.542258),
    'x_latency_0$': (0, 1.210133),
    'x_latency_1$': (0, 1.226670),
    'x_latency_2$': (0, 1.226670),
    'x_latency_3$': (0, 1.253752),
    'x_latency_4$': (0, 1.257164),
    'x_latency_5$': (0, 1.259484),
    'x_latency_6$': (0, 1.259484),
    'x_latency_7$': (0.083082, 1.259484),
    'x_next_size_0$': (0.111155, 0.181901),
    'x_next_size_1$': (0.277716, 0.450283),
    'x_next_size_2$': (0.487160, 0.709534),
    'x_next_size_3$': (0.728962, 1.076598),
    'x_next_size_4$': (1.139290, 1.728878),
    'x_next_size_5$': (1.919781, 2.395588),
    'x_remaining_chunks_0$': (0, 1),
}

discrete_inputs = {
    'x_prev_bitrate_0$': [v / 4300 for v in [300, 750, 1200, 1850, 2850, 4300]],
    'x_remaining_chunks_0$': [v / 48 for v in range(48)],
}

result_file = 'test_resilience_pensieve_case1'

### Experiment 1: attacker controls all features

In [ ]:
# Compute the full range of all variables that the attacker controls
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}

pprint(delta)

# Compute the value of epsilon
resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

# Print results
print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 1: all\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 2: attacker controls the previous bit rate feature

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if k != 'x_prev_bitrate_0$':
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)
print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 2: prev_bitrate\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 3: attacker controls the throughput and latency features

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if not 'throughput' in k and not 'latency' in k:
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 3: throughput + latency\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 4: attackers controls the buffer, next size and remaining chunk features

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if not 'buffer' in k and not 'next_size' in k and not 'remaining_chunks' in k:
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 4: buffer + next_size + remaining_chunks\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 5: attacker controls the buffer feature

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if not 'buffer' in k:
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 5: buffer\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 6: attacker controls the throughput feature

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if not 'throughput' in k:
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 6: throughput\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

### Experiment 7: attacker controls the throughput_7 and latency_7 features (most recent)

In [ ]:
delta = {k: bounds[k][1] - bounds[k][0] for k in bounds}
for k in delta:
    if not 'throughput_7' in k and not 'latency_7' in k:
        delta[k] = 0

pprint(delta)

resilience = search_perturbation_resilience(default_values,
                                            3,
                                            delta,
                                            discrete_inputs)

print('--------------------')
print('epsilon:', resilience)

with open(result_file, 'a') as _f:
    _f.write('exp 7: throughput_7 + latency_7\n')
    _f.write('epsilon:' + str(resilience) + '\n')

print("Individual bound per feature:")
for k in delta:
    print(f'{k}: ({round(default_values[k] - resilience * delta[k], 3)}, ' + \
                f'{round(default_values[k] + resilience * delta[k], 3)})')

## Case 2: Variable throughput, extracted from real traces

The values below represent the default input being tested in case 2.

The values are extracted from an experiment running the trace `norway_train_19` of the Pensieve testing dataset, at step 20.

To gather results in this test case, execute the box below then re-execute the seven experiments above.

In [ ]:
default_values = {
    'x_prev_bitrate_0$': 750/4300,
    'x_buffer_0$': 1.81199,
    'x_throughput_0$': 0.186370,
    'x_throughput_1$': 0.198280,
    'x_throughput_2$': 0.142516,
    'x_throughput_3$': 0.137576,
    'x_throughput_4$': 0.147882,
    'x_throughput_5$': 0.149303,
    'x_throughput_6$': 0.154915,
    'x_throughput_7$': 0.172726,
    'x_latency_0$': 0.556125,
    'x_latency_1$': 0.414377,
    'x_latency_2$': 0.647764,
    'x_latency_3$': 0.262277,
    'x_latency_4$': 0.252201,
    'x_latency_5$': 0.271659,
    'x_latency_6$': 0.226390,
    'x_latency_7$': 0.223747,
    'x_next_size_0$': 0.157812,
    'x_next_size_1$': 0.399894,
    'x_next_size_2$': 0.616206,
    'x_next_size_3$': 0.955231,
    'x_next_size_4$': 1.468126,
    'x_next_size_5$': 2.176469,
    'x_remaining_chunks_0$': 27/48,
}

bounds = {
    'x_prev_bitrate_0$': (300/4300, 4300/4300),
    'x_buffer_0$': (0.4, 3.267816),
    'x_throughput_0$': (0, 0.541045),
    'x_throughput_1$': (0, 0.541045),
    'x_throughput_2$': (0, 0.541045),
    'x_throughput_3$': (0, 0.541045),
    'x_throughput_4$': (0, 0.541045),
    'x_throughput_5$': (0, 0.542024),
    'x_throughput_6$': (0, 0.542086),
    'x_throughput_7$': (0.069767, 0.542258),
    'x_latency_0$': (0, 1.210133),
    'x_latency_1$': (0, 1.226670),
    'x_latency_2$': (0, 1.226670),
    'x_latency_3$': (0, 1.253752),
    'x_latency_4$': (0, 1.257164),
    'x_latency_5$': (0, 1.259484),
    'x_latency_6$': (0, 1.259484),
    'x_latency_7$': (0.083082, 1.259484),
    'x_next_size_0$': (0.111155, 0.181901),
    'x_next_size_1$': (0.277716, 0.450283),
    'x_next_size_2$': (0.487160, 0.709534),
    'x_next_size_3$': (0.728962, 1.076598),
    'x_next_size_4$': (1.139290, 1.728878),
    'x_next_size_5$': (1.919781, 2.395588),
    'x_remaining_chunks_0$': (0, 1),
}

discrete_inputs = {
    'x_prev_bitrate_0$': [v / 4300 for v in [300, 750, 1200, 1850, 2850, 4300]],
    'x_remaining_chunks_0$': [v / 48 for v in range(48)],
}

result_file = 'test_resilience_pensieve_case2'

## Experiment 2: Robustness against missing features

This experiment aims to test if the Pensieve agent continues to return the same result as the original when some features are "missing", i.e. if they can take any value in the allowed range.

### Implementation

This experiment is implemented by encoding the bounds as the full variable range for variables that are in the testing group (the missing variables), and encoding the other variables as constrants.

The result of the experiments is written to the file `test_missing_pensieve_caseX`.

In [ ]:
def test_resilience(default_values, expected_br, test_feature, test_feature_bounds):
    # returns true if the missing feature does not cause a different decision
    
    # Encoding of the network
    exp = pensieve.PropertyExplainer()
    exp.encode_input_layers()
    exp.encode_hidden_layers()
    
    for var in default_values:
        if var not in test_feature:
            exp.encode_tighter_bounds(var, default_values[var], default_values[var]) # set to constant
        else:
            exp.encode_tighter_bounds(var, *test_feature_bounds[var])
            
    exp.encode_expected_output(expected_br, inverted=True)
    
    exp.find_solutions(stop_after_one=True)
    
    #return len(exp.solutions) == 0, exp
    return len(exp.solutions) == 0

## Case 1: Constant throughput of 1850Kbps

The values below represent the default input being tested in case 1.

In [ ]:
default_values = {
    'x_prev_bitrate_0$': 1850/4300, # expect 1850 Kbps
    'x_buffer_0$': 1.573854,
    'x_throughput_0$': 0.23125, # fixed 1850 Kbps goodput
    'x_throughput_1$': 0.23125,
    'x_throughput_2$': 0.23125,
    'x_throughput_3$': 0.23125,
    'x_throughput_4$': 0.23125,
    'x_throughput_5$': 0.23125,
    'x_throughput_6$': 0.23125,
    'x_throughput_7$': 0.23125,
    'x_latency_0$': 0.504, # next_size_3 / throughput / 8
    'x_latency_1$': 0.504,
    'x_latency_2$': 0.504,
    'x_latency_3$': 0.504,
    'x_latency_4$': 0.504,
    'x_latency_5$': 0.504,
    'x_latency_6$': 0.504,
    'x_latency_7$': 0.504,
    'x_next_size_0$': 0.153233,
    'x_next_size_1$': 0.380457,
    'x_next_size_2$': 0.605353,
    'x_next_size_3$': 0.931950,
    'x_next_size_4$': 1.437556,
    'x_next_size_5$': 2.158495,
    'x_remaining_chunks_0$': 24/48,
}

bounds = {
    'x_prev_bitrate_0$': (300/4300, 4300/4300),
    'x_buffer_0$': (0.4, 3.267816),
    'x_throughput_0$': (0, 0.541045),
    'x_throughput_1$': (0, 0.541045),
    'x_throughput_2$': (0, 0.541045),
    'x_throughput_3$': (0, 0.541045),
    'x_throughput_4$': (0, 0.541045),
    'x_throughput_5$': (0, 0.542024),
    'x_throughput_6$': (0, 0.542086),
    'x_throughput_7$': (0.069767, 0.542258),
    'x_latency_0$': (0, 1.210133),
    'x_latency_1$': (0, 1.226670),
    'x_latency_2$': (0, 1.226670),
    'x_latency_3$': (0, 1.253752),
    'x_latency_4$': (0, 1.257164),
    'x_latency_5$': (0, 1.259484),
    'x_latency_6$': (0, 1.259484),
    'x_latency_7$': (0.083082, 1.259484),
    'x_next_size_0$': (0.111155, 0.181901),
    'x_next_size_1$': (0.277716, 0.450283),
    'x_next_size_2$': (0.487160, 0.709534),
    'x_next_size_3$': (0.728962, 1.076598),
    'x_next_size_4$': (1.139290, 1.728878),
    'x_next_size_5$': (1.919781, 2.395588),
    'x_remaining_chunks_0$': (0, 1),
}

result_file = 'test_missing_pensieve_case1'

### Experiment 1: all throughput values are missing

In [ ]:
test_vars = [f'x_throughput_{i}$' for i in range(8)]

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 1: throughput\n')
    _f.write('resilient:' + str(result) + '\n')

### Experiment 2: all latency values are missing

In [ ]:
test_vars = [f'x_latency_{i}$' for i in range(4, 8)]

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 2: latency\n')
    _f.write('resilient:' + str(result) + '\n')

### Experiment 2.1: the 4 most recent latency values are missing

In [ ]:
test_vars = ['x_latency_4$', 'x_latency_5$', 'x_latency_6$', 'x_latency_7$']

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 2: latency\n')
    _f.write('resilient:' + str(result) + '\n')

### Experiment 3: the most recent throughput value is missing

In [ ]:
test_vars = ['x_throughput_7$']

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 3: throughput_7\n')
    _f.write('resilient:' + str(result) + '\n')

### Experiment 4: all next size values are missing

In [ ]:
test_vars = [f'x_next_size_{i}$' for i in range(6)]

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 4: next_size\n')
    _f.write('resilient:' + str(result) + '\n')

### Experiment 5: the previous selected bit rate value is missing

**Note:** The value can become anything in the range of possible values, which is still discrete in this case.

In [ ]:
test_vars = ['x_prev_bitrate_0$']

result = test_resilience(default_values, 3, test_vars, bounds)

print('-------------------------------------')
print('resilient to missing feature:', result)

with open(result_file, 'a') as _f:
    _f.write('exp 5: prev_bitrate\n')
    _f.write('resilient:' + str(result) + '\n')

## Case 2: Variable throughput, extracted from real traces

The values below represent the default input being tested in case 2.

The values are extracted from an experiment running the trace `norway_train_19` of the Pensieve testing dataset, at step 20.

To gather results in this test case, execute the box below then re-execute the six experiments above.

In [ ]:
default_values = {
    'x_prev_bitrate_0$': 750/4300,
    'x_buffer_0$': 1.81199,
    'x_throughput_0$': 0.186370,
    'x_throughput_1$': 0.198280,
    'x_throughput_2$': 0.142516,
    'x_throughput_3$': 0.137576,
    'x_throughp:q:qut_4$': 0.147882,
    'x_throughput_5$': 0.149303,
    'x_throughput_6$': 0.154915,
    'x_throughput_7$': 0.172726,
    'x_latency_0$': 0.556125,
    'x_latency_1$': 0.414377,
    'x_latency_2$': 0.647764,
    'x_latency_3$': 0.262277,
    'x_latency_4$': 0.252201,
    'x_latency_5$': 0.271659,
    'x_latency_6$': 0.226390,
    'x_latency_7$': 0.223747,
    'x_next_size_0$': 0.157812,
    'x_next_size_1$': 0.399894,
    'x_next_size_2$': 0.616206,
    'x_next_size_3$': 0.955231,
    'x_next_size_4$': 1.468126,
    'x_next_size_5$': 2.176469,
    'x_remaining_chunks_0$': 27/48,
}

bounds = {
    'x_prev_bitrate_0$': (300/4300, 4300/4300),
    'x_buffer_0$': (0.4, 3.267816),
    'x_throughput_0$': (0, 0.541045),
    'x_throughput_1$': (0, 0.541045),
    'x_throughput_2$': (0, 0.541045),
    'x_throughput_3$': (0, 0.541045),
    'x_throughput_4$': (0, 0.541045),
    'x_throughput_5$': (0, 0.542024),
    'x_throughput_6$': (0, 0.542086),
    'x_throughput_7$': (0.069767, 0.542258),
    'x_latency_0$': (0, 1.210133),
    'x_latency_1$': (0, 1.226670),
    'x_latency_2$': (0, 1.226670),
    'x_latency_3$': (0, 1.253752),
    'x_latency_4$': (0, 1.257164),
    'x_latency_5$': (0, 1.259484),
    'x_latency_6$': (0, 1.259484),
    'x_latency_7$': (0.083082, 1.259484),
    'x_next_size_0$': (0.111155, 0.181901),
    'x_next_size_1$': (0.277716, 0.450283),
    'x_next_size_2$': (0.487160, 0.709534),
    'x_next_size_3$': (0.728962, 1.076598),
    'x_next_size_4$': (1.139290, 1.728878),
    'x_next_size_5$': (1.919781, 2.395588),
    'x_remaining_chunks_0$': (0, 1),
}

result_file = 'test_missing_pensieve_case2'

## Experiment 3: Decision Boundaries

This experiment aims to show how constraints can be used to extract the decisions boundaries from Pensieve.

### Implementation

This experiment is implemented by setting all input values except the two axes of our plot to some constant value, the values corresponding to the axes to the desired range, and the output to the class we are looking for.

The output is written to files `mapping_outputs_throughput_buffer_with_prev_XXX.pdf`.

## With previous bit rate of 300Kbps

In [ ]:
def plot_area(free_vars, expected_br):
    # Encoding of the network
    exp = pensieve.PropertyExplainer()
    exp.encode_input_layers()
    exp.encode_hidden_layers()

    input_names = list()
    for layer in exp.layers:
        input_names.extend(layer.input_names)
    
    default_values = {
        'x_prev_bitrate_0$': 300/4300, # changed
        'x_buffer_0$': 1.573854,
        'x_throughput_0$': 0.09375,
        'x_throughput_1$': 0.09375,
        'x_throughput_2$': 0.09375,
        'x_throughput_3$': 0.09375,
        'x_throughput_4$': 0.09375,
        'x_throughput_5$': 0.09375,
        'x_throughput_6$': 0.09375,
        'x_throughput_7$': 0.155318,
        'x_latency_0$': 0.20431,
        'x_latency_1$': 0.20431,
        'x_latency_2$': 0.20431,
        'x_latency_3$': 0.20431,
        'x_latency_4$': 0.20431,
        'x_latency_5$': 0.20431,
        'x_latency_6$': 0.20431,
        'x_latency_7$': 0.20431,
        'x_next_size_0$': 0.153233,
        'x_next_size_1$': 0.380457,
        'x_next_size_2$': 0.605353,
        'x_next_size_3$': 0.931950,
        'x_next_size_4$': 1.437556,
        'x_next_size_5$': 2.158495,
        'x_remaining_chunks_0$': 24/48,
    }
    
    # Encode property
    for var in free_vars:
        default_values.pop(var)
    
    exp.encode_constant_values(default_values)
    for var, bounds in free_vars.items():
        lb, ub = bounds
        exp.encode_tighter_bounds(var, lb, ub)

    exp.encode_expected_output(expected_br)
    
    # Find solution
    exp.find_solutions()
    
    all_polytopes = list()

    for i in range(len(exp.solutions)):
        params = exp.find_linear_equation(i, free_vars)
        exp.find_simplified_params(params, free_vars)
    
        constr = exp.find_constraints(params, free_vars)
        constr.append((1, 0, -min(list(free_vars.values())[0])))
        constr.append((-1, 0, max(list(free_vars.values())[0])))
        constr.append((0, 1, -min(list(free_vars.values())[1])))
        constr.append((0, -1, max(list(free_vars.values())[1])))
        
        points = exp.find_polytope(constr)
        all_polytopes.append(points)
        
    return all_polytopes

In [ ]:
free_vars = OrderedDict({
    'x_buffer_0$': (0.4, 1.6),
    'x_throughput_7$': (0/8000, 2000/8000)
})

areas = list()

areas.append(plot_area(free_vars, 0))
areas.append(plot_area(free_vars, 1))

In [ ]:
# Plot results
for all_polytopes, color in zip(areas, 'rgbcym'):
    for points in all_polytopes:
        plt.plot(10 * np.array(points[:, 0]), 8000 * np.array(points[:, 1]), 'k', linewidth=.3)
        plt.fill(10 * np.array(points[:, 0]), 8000 * np.array(points[:, 1]), color)
        
#plt.xlabel(list(free_vars)[0])
plt.xlabel('Buffer [s]')
#plt.ylabel(list(free_vars)[1])
plt.ylabel('Throughput [Kbps]')

plt.xlim(10 * np.array(free_vars[list(free_vars)[0]]))
plt.ylim(8000 * np.array(free_vars[list(free_vars)[1]]))

plt.plot([13], [1160], 'c*', markersize=13)
plt.plot([6], [800], 'c*', markersize=13)
plt.plot([10], [1000], 'y*', markersize=13)

plt.savefig('mapping_outputs_throughput_buffer_with_prev_300.pdf')
plt.show()

## With previous bit rate of 750Kbps

In [ ]:
def plot_area(free_vars, expected_br):
    # Encoding of the network
    exp = pensieve.PropertyExplainer()
    exp.encode_input_layers()
    exp.encode_hidden_layers()

    input_names = list()
    for layer in exp.layers:
        input_names.extend(layer.input_names)
    
    default_values = {
        'x_prev_bitrate_0$': 750/4300, # changed
        'x_buffer_0$': 1.573854,
        'x_throughput_0$': 0.09375,
        'x_throughput_1$': 0.09375,
        'x_throughput_2$': 0.09375,
        'x_throughput_3$': 0.09375,
        'x_throughput_4$': 0.09375,
        'x_throughput_5$': 0.09375,
        'x_throughput_6$': 0.09375,
        'x_throughput_7$': 0.155318,
        'x_latency_0$': 0.507276,
        'x_latency_1$': 0.507276,
        'x_latency_2$': 0.507276,
        'x_latency_3$': 0.507276,
        'x_latency_4$': 0.507276,
        'x_latency_5$': 0.507276,
        'x_latency_6$': 0.507276,
        'x_latency_7$': 0.507276,
        'x_next_size_0$': 0.153233,
        'x_next_size_1$': 0.380457,
        'x_next_size_2$': 0.605353,
        'x_next_size_3$': 0.931950,
        'x_next_size_4$': 1.437556,
        'x_next_size_5$': 2.158495,
        'x_remaining_chunks_0$': 24/48,
    }
    
    # Encode property
    for var in free_vars:
        default_values.pop(var)
    
    exp.encode_constant_values(default_values)
    for var, bounds in free_vars.items():
        lb, ub = bounds
        exp.encode_tighter_bounds(var, lb, ub)

    exp.encode_expected_output(expected_br)
    
    # Find solution
    exp.find_solutions()
    
    all_polytopes = list()

    for i in range(len(exp.solutions)):
        params = exp.find_linear_equation(i, free_vars)
        exp.find_simplified_params(params, free_vars)
    
        constr = exp.find_constraints(params, free_vars)
        constr.append((1, 0, -min(list(free_vars.values())[0])))
        constr.append((-1, 0, max(list(free_vars.values())[0])))
        constr.append((0, 1, -min(list(free_vars.values())[1])))
        constr.append((0, -1, max(list(free_vars.values())[1])))
        
        points = exp.find_polytope(constr)
        all_polytopes.append(points)
        
    return all_polytopes

In [ ]:
free_vars = OrderedDict({
    'x_buffer_0$': (0.4, 1.6),
    'x_throughput_7$': (0/8000, 2000/8000)
})

areas = list()

areas.append(plot_area(free_vars, 0))
areas.append(plot_area(free_vars, 1))

In [ ]:
# Plot results
for all_polytopes, color in zip(areas, 'rgbcym'):
    for points in all_polytopes:
        plt.plot(10 * np.array(points[:, 0]), 8000 * np.array(points[:, 1]), 'k', linewidth=.3)
        plt.fill(10 * np.array(points[:, 0]), 8000 * np.array(points[:, 1]), color)
        
#plt.xlabel(list(free_vars)[0])
plt.xlabel('Buffer [s]')
#plt.ylabel(list(free_vars)[1])
plt.ylabel('Throughput [Kbps]')

plt.xlim(10 * np.array(free_vars[list(free_vars)[0]]))
plt.ylim(8000 * np.array(free_vars[list(free_vars)[1]]))

plt.plot([13], [1160], 'c*', markersize=13)
plt.plot([6], [800], 'c*', markersize=13)
plt.plot([10], [1000], 'y*', markersize=13)

plt.savefig('mapping_outputs_throughput_buffer_with_prev_750.pdf')
plt.show()

## Experiment 4: Verifying specific properties

This experiment aims to show how constraints can be used to verify domain-specific properties. In this example, we are looking for all values where the Pensieve agent will select a bit rate of 1200Kbps as the choice with the highest value.

### First example: Trying to plot the areas where 1200Kbps is selected

This example tests with only the buffer and throughput as "free" variables, while the other variables are set to some constant.

Expected: result is empty.

In [ ]:
def plot_area(free_vars, expected_br):
    # Encoding of the network
    exp = pensieve.PropertyExplainer()
    exp.encode_input_layers()
    exp.encode_hidden_layers()

    input_names = list()
    for layer in exp.layers:
        input_names.extend(layer.input_names)
    
    default_values = {
        'x_prev_bitrate_0$': 1200/4300,
        'x_buffer_0$': 1.573854,
        'x_throughput_0$': 0.15,
        'x_throughput_1$': 0.15,
        'x_throughput_2$': 0.15,
        'x_throughput_3$': 0.15,
        'x_throughput_4$': 0.15,
        'x_throughput_5$': 0.15,
        'x_throughput_6$': 0.15,
        'x_throughput_7$': 0.15,
        'x_latency_0$': 0.50446,
        'x_latency_1$': 0.50446,
        'x_latency_2$': 0.50446,
        'x_latency_3$': 0.50446,
        'x_latency_4$': 0.50446,
        'x_latency_5$': 0.50446,
        'x_latency_6$': 0.50446,
        'x_latency_7$': 0.50446,
        'x_next_size_0$': 0.153233,
        'x_next_size_1$': 0.380457,
        'x_next_size_2$': 0.605353,
        'x_next_size_3$': 0.931950,
        'x_next_size_4$': 1.437556,
        'x_next_size_5$': 2.158495,
        'x_remaining_chunks_0$': 24/48,
    }
    
    # Encode property
    for var in free_vars:
        default_values.pop(var)
    
    exp.encode_constant_values(default_values)
    for var, bounds in free_vars.items():
        lb, ub = bounds
        exp.encode_tighter_bounds(var, lb, ub)

    exp.encode_expected_output(expected_br)
    
    # Find solution
    exp.find_solutions()
    
    all_polytopes = list()

    for i in range(len(exp.solutions)):
        params = exp.find_linear_equation(i, free_vars)
        exp.find_simplified_params(params, free_vars)
    
        constr = exp.find_constraints(params, free_vars)
        constr.append((1, 0, -min(list(free_vars.values())[0])))
        constr.append((-1, 0, max(list(free_vars.values())[0])))
        constr.append((0, 1, -min(list(free_vars.values())[1])))
        constr.append((0, -1, max(list(free_vars.values())[1])))
        
        points = exp.find_polytope(constr)
        all_polytopes.append(points)
        
    return all_polytopes

In [ ]:
free_vars = OrderedDict({
    'x_buffer_0$': (0.4, 3.2),
    'x_throughput_7$': (0/8000, 3000/8000)
})

areas = list()

areas.append(plot_area(free_vars, 2))

In [ ]:
# Plot results
for all_polytopes, color in zip(areas, 'rgbcym'):
    for points in all_polytopes:
        plt.plot(points[:, 0], points[:, 1])
        plt.fill(points[:, 0], points[:, 1], color)
        
plt.xlabel(list(free_vars)[0])
plt.ylabel(list(free_vars)[1])

plt.xlim(free_vars[list(free_vars)[0]])
plt.ylim(free_vars[list(free_vars)[1]])

plt.show()

### Second example: relaxing more variables

In this example, we have five different variables that became free.

**Note:** Depending on the number and range of variables being tested, the verification can potentially run for a long time. If the experiment takes too long to conclude, consider reducing the search space. 

In [ ]:
exp = pensieve.PropertyExplainer()
exp.encode_input_layers()
exp.encode_hidden_layers()

default_values = {
    'x_prev_bitrate_0$': 750/4300,
    'x_buffer_0$': 1.573854,
    'x_throughput_0$': 0.155318,
    'x_throughput_1$': 0.155318,
    'x_throughput_2$': 0.155318,
    'x_throughput_3$': 0.155318,
    'x_throughput_4$': 0.155318,
    'x_throughput_5$': 0.155318,
    'x_throughput_6$': 0.155318,
    'x_throughput_7$': 0.155318,
    'x_latency_0$': 0.374323,
    'x_latency_1$': 0.374323,
    'x_latency_2$': 0.374323,
    'x_latency_3$': 0.374323,
    'x_latency_4$': 0.374323,
    'x_latency_5$': 0.374323,
    'x_latency_6$': 0.374323,
    'x_latency_7$': 0.374323,
    'x_next_size_0$': 0.153233,
    'x_next_size_1$': 0.380457,
    'x_next_size_2$': 0.605353,
    'x_next_size_3$': 0.931950,
    'x_next_size_4$': 1.437556,
    'x_next_size_5$': 2.158495,
    'x_remaining_chunks_0$': 24/48,
}

free_vars = OrderedDict({
    'x_buffer_0$': (0.4, 3.2),
    'x_throughput_7$': (0/8000, 3000/8000),
    'x_prev_bitrate_0$': (750/4300, 1850/4300),
    'x_latency_7$': (0.0, 1.2),
    'x_remaining_chunks_0$': (0/48, 1),
})

exp.encode_expected_output(2)
    
# Find solution
exp.find_solutions(stop_after_one=True)